In [7]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [3]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    return df

In [38]:
train = load_df("data/train.csv")
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [13]:
# agg_df['fullVisitorId'].plot(kind='bar')



In [40]:
# dropping single-valued columns
# note that nunique() will assign value of '1' to columns
# which have 1 value + NaNs, so nunique will yield 
# false positives

# dropping socialEngagementType
# dropping device.browserSize
# dropping device.browserVersion
# dropping device.flashVersion
# dropping device.language
# dropping device.mobileDeviceBranding
# dropping device.mobileDeviceInfo
# dropping device.mobileDeviceMarketingName
# dropping device.mobileDeviceModel
# dropping device.mobileInputSelector
# dropping device.operatingSystemVersion
# dropping device.screenColors
# dropping device.screenResolution
# dropping geoNetwork.cityId
# dropping geoNetwork.latitude
# dropping geoNetwork.longitude
# dropping geoNetwork.networkLocation
# dropping totals.visits
# dropping trafficSource.adwordsClickInfo.criteriaParameters

for col in train.columns:
    if len(train[col].unique()) < 2:
        train.drop(col, axis=1, inplace=True)
        print("dropping " + col)

dropping socialEngagementType
dropping device.browserSize
dropping device.browserVersion
dropping device.flashVersion
dropping device.language
dropping device.mobileDeviceBranding
dropping device.mobileDeviceInfo
dropping device.mobileDeviceMarketingName
dropping device.mobileDeviceModel
dropping device.mobileInputSelector
dropping device.operatingSystemVersion
dropping device.screenColors
dropping device.screenResolution
dropping geoNetwork.cityId
dropping geoNetwork.latitude
dropping geoNetwork.longitude
dropping geoNetwork.networkLocation
dropping totals.visits
dropping trafficSource.adwordsClickInfo.criteriaParameters


In [44]:
train.dtypes

channelGrouping                                 object
date                                             int64
fullVisitorId                                   object
sessionId                                       object
visitId                                          int64
visitNumber                                      int64
visitStartTime                                   int64
device.browser                                  object
device.deviceCategory                           object
device.isMobile                                   bool
device.operatingSystem                          object
geoNetwork.city                                 object
geoNetwork.continent                            object
geoNetwork.country                              object
geoNetwork.metro                                object
geoNetwork.networkDomain                        object
geoNetwork.region                               object
geoNetwork.subContinent                         object
totals.bou

In [87]:
tmp = train['trafficSource.keyword'].apply(type).value_counts(dropna=False)
str(tmp[1:2])

"<class 'str'>    400724\nName: trafficSource.keyword, dtype: int64"

In [68]:

def col_type_counts(df):
    col_types = {}
    for col in df.columns:
        col_types[col] = df[col].apply(type).value_counts()
    return col_types
        
print(col_type_counts(train))

{'channelGrouping': <class 'str'>    903653
Name: channelGrouping, dtype: int64, 'date': <class 'int'>    903653
Name: date, dtype: int64, 'fullVisitorId': <class 'str'>    903653
Name: fullVisitorId, dtype: int64, 'sessionId': <class 'str'>    903653
Name: sessionId, dtype: int64, 'visitId': <class 'int'>    903653
Name: visitId, dtype: int64, 'visitNumber': <class 'int'>    903653
Name: visitNumber, dtype: int64, 'visitStartTime': <class 'int'>    903653
Name: visitStartTime, dtype: int64, 'device.browser': <class 'str'>    903653
Name: device.browser, dtype: int64, 'device.deviceCategory': <class 'str'>    903653
Name: device.deviceCategory, dtype: int64, 'device.isMobile': <class 'bool'>    903653
Name: device.isMobile, dtype: int64, 'device.operatingSystem': <class 'str'>    903653
Name: device.operatingSystem, dtype: int64, 'geoNetwork.city': <class 'str'>    903653
Name: geoNetwork.city, dtype: int64, 'geoNetwork.continent': <class 'str'>    903653
Name: geoNetwork.continent, dt

In [49]:
# train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')
# gdf = train_df.groupby("fullVisitorId")["totals.transactionRevenue"].sum().reset_index()

# plt.figure(figsize=(8,6))
# plt.scatter(range(gdf.shape[0]), np.sort(np.log1p(gdf["totals.transactionRevenue"].values)))
# plt.xlabel('index', fontsize=12)
# plt.ylabel('TransactionRevenue', fontsize=12)
# plt.show()

In [27]:
# The main types stored in pandas objects are float, int, bool, 
# datetime64[ns] and datetime64[ns, tz], timedelta[ns], category and object.

channelGrouping -> string
date -> date
bounces -> manual, nan, 1
totals.hits -> int64
totals.newVisits -> manual array(['1', nan], dtype=object)
totals.pageviews -> int64
totals.transactionRevenue -> nan + ints -> int64



,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,...,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google


In [30]:
train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.hits', 'totals.pageviews',
       'totals.transactionRevenue', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.medium',
       'trafficSource.referralPath', 'trafficSource.source'],
      dtype='object')